In [1]:
import scanpy as sc 
import pandas as pd
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 70)  # show all columns
pd.set_option('display.width', None)        # don’t wrap columns

In [2]:
adata= sc.read_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_filtered_with_metadata.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 1579208 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'Age_pf', 'Anatomical_region_pf', 'BCR_library_id', 'Comment[library construction]', 'Comment[technical replicate group]', 'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf', 'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf', 'Processing_method', 'Processing_method_pf', 'S

In [4]:
# list exactly the obs columns you want to remove
cols_to_remove = [
    'Age_pf', 'Anatomical_region_pf', 'BCR_library_id',
    'Comment[library construction]', 'Comment[technical replicate group]',
    'Donor_pf', 'End_Bias', 'Factor Value[facs sorting]', 'File_pf',
    'Method_pf', 'N_cells', 'N_cells_pf', 'Organ_Orig', 'Organ_pf',
    'Processing_method', 'Processing_method_pf', 'Sample', 'Sample_ID_Copy',
    'Sample_pf', 'Sampling_Site', 'Sex', 'Sex_pf', 'Sort_id_pf',
    'Termination_method_pf', 'Unnamed: 14', 'abTCR_library_id_pf',
    'gdTCR_library_id_pf', 'Organ.1',
]

# drop them all at once from adata.obs
adata.obs.drop(columns=cols_to_remove, inplace=True)

# sanity check: ensure they’re gone
print([c for c in cols_to_remove if c in adata.obs.columns])
# should print an empty list


[]


In [5]:
adata

AnnData object with n_obs × n_vars = 1579208 × 36601
    obs: 'barcode_raw', 'Sanger_ID', 'Cell_ID', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'log1p_total_counts_rb', 'pct_counts_rb', 'total_counts_outlier', 'gene_counts_outlier', 'mt_outlier', 'Organ', '10X_Chemistry', 'Accession_ID', 'Age', 'Age(PCW)', 'Dataset', 'Donor_ID', 'FACS_Sorting', 'Anatomical_Site', 'Method', 'Multiplexed', 'Expected_Donors', 'Organism', 'Run_ID', 'Study', 'Technology', 'predicted_labels', 'over_clustering', 'majority_voting', 'predicted_labels_Level1', 'predicted_labels_Level2', 'majority_voting_Level1', 'majority_voting_Level2', 'mt_high_outlier', 'QC'

In [6]:
doublets= pd.read_csv('/lustre/scratch126/cellgen/haniffa/sm54/BoneAtlasProject/data/combined_scrublet_scores_raw_bone_atlas.csv')

/tmp/ipykernel_3748640/1352984862.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  doublets= pd.read_csv('/lustre/scratch126/cellgen/haniffa/sm54/BoneAtlasProject/data/combined_scrublet_scores_raw_bone_atlas.csv')


In [8]:
doublets.dtypes

Cell_ID                     object
scrublet_score             float64
predicted_doublets          object
scrublet_cluster_score     float64
bh_pval                    float64
predicted_doublets_pval       bool
dtype: object

In [9]:
# 2) Prepare doublets_df: set its index to Cell_ID so the join lines up
doublets_indexed = doublets.set_index('Cell_ID')

# 3) Join all columns from doublets_df into adata.obs, preserving all cells
adata.obs = adata.obs.join(doublets_indexed, how='left')

In [10]:
# 4) (Optional) Verify that new columns appeared
print([c for c in doublets.columns if c != 'Cell_ID'])

['scrublet_score', 'predicted_doublets', 'scrublet_cluster_score', 'bh_pval', 'predicted_doublets_pval']


In [11]:
# List of columns to rename
cols_to_rename = [
    'scrublet_score',
    'predicted_doublets',
    'scrublet_cluster_score',
    'bh_pval',
    'predicted_doublets_pval'
]

# Build the rename mapping
rename_dict = {col: f"{col}_unfiltered" for col in cols_to_rename}

# Rename in place in adata.obs
adata.obs.rename(columns=rename_dict, inplace=True)


In [12]:
(adata.obs.head())

,barcode_raw,Sanger_ID,Cell_ID,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_rb,log1p_total_counts_rb,pct_counts_rb,total_counts_outlier,gene_counts_outlier,mt_outlier,Organ,10X_Chemistry,Accession_ID,Age,Age(PCW),Dataset,Donor_ID,FACS_Sorting,Anatomical_Site,Method,Multiplexed,Expected_Donors,Organism,Run_ID,Study,Technology,predicted_labels,over_clustering,majority_voting,predicted_labels_Level1,predicted_labels_Level2,majority_voting_Level1,majority_voting_Level2,mt_high_outlier,QC,scrublet_score_unfiltered,predicted_doublets_unfiltered,scrublet_cluster_score_unfiltered,bh_pval_unfiltered,predicted_doublets_pval_unfiltered
Cell_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CGCAATGTCAGCAAGA-HCA_BN_F12482363,CGCAATGTCAGCAAGA,HCA_BN_F12482363,CGCAATGTCAGCAAGA-HCA_BN_F12482363,336,5.820083,524,6.263398,39.885496,54.961832,74.045802,100.0,2,1.098612,0.381679,61,4.127134,11.641221,False,False,False,Calvaria,10x_multiome,E-MTAB-14385,8.5,8.5,Bone_Atlas,2361,Total,calvaria,10X_multiome,No,1,Homo sapiens,HCA_BN_F12482363_and_HCA_BN_F12477275,To_et_al_2024,10X_multiome,Skull erythroid cells,19,Skull erythroid cells,Erythroid,Erythroid,Erythroid,Erythroid,False,Pass,0.377787,False,0.377787,0.006197,True
TAATCACCAATTGACT-HCA_BN_F12482363,TAATCACCAATTGACT,HCA_BN_F12482363,TAATCACCAATTGACT-HCA_BN_F12482363,388,5.963579,527,6.269096,32.827324,45.351044,64.326376,100.0,0,0.000000,0.000000,111,4.718499,21.062619,False,False,False,Calvaria,10x_multiome,E-MTAB-14385,8.5,8.5,Bone_Atlas,2361,Total,calvaria,10X_multiome,No,1,Homo sapiens,HCA_BN_F12482363_and_HCA_BN_F12477275,To_et_al_2024,10X_multiome,Definitive erythrocytes,48,neural crest,Erythroid,Erythroid,Mesenchymal,Mesenchymal,False,Pass,0.099246,False,0.377787,0.006197,True
CAAGTAACATGACTAT-HCA_BN_F12482363,CAAGTAACATGACTAT,HCA_BN_F12482363,CAAGTAACATGACTAT-HCA_BN_F12482363,396,5.983936,517,6.249975,27.272727,42.746615,62.088975,100.0,0,0.000000,0.000000,60,4.110874,11.605416,False,False,False,Calvaria,10x_multiome,E-MTAB-14385,8.5,8.5,Bone_Atlas,2361,Total,calvaria,10X_multiome,No,1,Homo sapiens,HCA_BN_F12482363_and_HCA_BN_F12477275,To_et_al_2024,10X_multiome,Coelomic epithelial cells,10,Perisuture cels,Stroma,Epithelial,Mesenchymal,Mesenchymal,False,Pass,0.353663,False,0.422427,0.000030,True
AAAGCCGCATGTTGCA-HCA_BN_F12482363,AAAGCCGCATGTTGCA,HCA_BN_F12482363,AAAGCCGCATGTTGCA-HCA_BN_F12482363,305,5.723585,533,6.280396,52.157598,61.538462,80.300188,100.0,2,1.098612,0.375235,86,4.465908,16.135084,False,False,False,Calvaria,10x_multiome,E-MTAB-14385,8.5,8.5,Bone_Atlas,2361,Total,calvaria,10X_multiome,No,1,Homo sapiens,HCA_BN_F12482363_and_HCA_BN_F12477275,To_et_al_2024,10X_multiome,neural crest,19,Skull erythroid cells,Mesenchymal,Mesenchymal,Erythroid,Erythroid,False,Pass,0.394934,False,0.377787,0.006197,True
TGGCGGTTCTCCTCAA-HCA_BN_F12482363,TGGCGGTTCTCCTCAA,HCA_BN_F12482363,TGGCGGTTCTCCTCAA-HCA_BN_F12482363,386,5.958425,531,6.276643,32.203390,46.139360,64.971751,100.0,12,2.564949,2.259887,112,4.727388,21.092279,False,False,False,Calvaria,10x_multiome,E-MTAB-14385,8.5,8.5,Bone_Atlas,2361,Total,calvaria,10X_multiome,No,1,Homo sapiens,HCA_BN_F12482363_and_HCA_BN_F12477275,To_et_al_2024,10X_multiome,neural crest,21,neural crest,Mesenchymal,Mesenchymal,Mesenchymal,Mesenchymal,False,Pass,0.626226,False,0.272867,0.942693,False


In [23]:
adata.obs['predicted_doublets_pval_unfiltered']=adata.obs['predicted_doublets_pval_unfiltered'].astype(str)
adata.obs['predicted_doublets_unfiltered']=adata.obs['predicted_doublets_unfiltered'].astype(str)

In [15]:
adata.obs['predicted_doublets_pval_unfiltered'].value_counts()

predicted_doublets_pval_unfiltered
False    1559679
True       19529
Name: count, dtype: int64

# add demultiplexed barcodes

In [3]:
import pandas as pd

In [4]:
dmx_barcodes= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_barcodes.csv')

/tmp/ipykernel_1315/835668580.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dmx_barcodes= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_barcodes.csv')


In [ ]:
dmx_barcodes= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/')

In [5]:
dmx_barcodes


,Unnamed: 0,Cell_ID,barcode,status,assignment,Sample_ID,Donor,Sample_ID_pcw13,Donor_ID,pcw,...,Celltype_fig1,pcw_published,brc_code,anatomical_site,Sample_ID_old_anno,Donor_old_anno,ANATOMICAL_SITE,PCW,Donor_final,Age_final
0,0,AAACAGCCAACTGGGA-HCA_BN_F13177434,AAACAGCCAACTGGGA,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434,NaN,NaN,NaN,...,Periosteal Dura,10.3,2365.0,calvaria,HCA_BN_F13177434,2365.0,calvaria,10.3,2365.0,10.3
1,1,AAACAGCCAATAACCT-HCA_BN_F13177434,AAACAGCCAATAACCT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,...,DermFIB1,11.2,2372.0,calvaria,HCA_BN_F13177434,2372.0,calvaria,11.2,2372.0,11.2
2,2,AAACAGCCACAGGAAT-HCA_BN_F13177434,AAACAGCCACAGGAAT,singlet,0,HCA_BN_F13177434,0-HCA_BN_F13177434,NaN,NaN,NaN,...,Periosteal Dura,10.3,2365.0,calvaria,HCA_BN_F13177434,2365.0,calvaria,10.3,2365.0,10.3
3,3,AAACAGCCAGTAAAGC-HCA_BN_F13177434,AAACAGCCAGTAAAGC,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2372.0,11.2
4,4,AAACAGCCAGTTGCGT-HCA_BN_F13177434,AAACAGCCAGTTGCGT,singlet,1,HCA_BN_F13177434,1-HCA_BN_F13177434,NaN,NaN,NaN,...,HHIP+PreOB,11.2,2372.0,calvaria,HCA_BN_F13177434,2372.0,calvaria,11.2,2372.0,11.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191955,191955,TTTGTGAAGGGTGGAT-HCA_BN_F13109719,TTTGTGAAGGGTGGAT,doublet,1/0,HCA_BN_F13109719,1/0-HCA_BN_F13109719,NaN,NaN,NaN,...,ChondroPro2,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2,2372.0,11.2
191956,191956,TTTGTGGCAGAAATTG-HCA_BN_F13109719,TTTGTGGCAGAAATTG,singlet,0,HCA_BN_F13109719,0-HCA_BN_F13109719,NaN,NaN,NaN,...,SynFIB,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2,2372.0,11.2
191957,191957,TTTGTTGGTGATGGCT-HCA_BN_F13109719,TTTGTTGGTGATGGCT,doublet,1/0,HCA_BN_F13109719,1/0-HCA_BN_F13109719,NaN,NaN,NaN,...,ChondroPro1,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2,2372.0,11.2
191958,191958,TTTGTTGGTTCAAGCA-HCA_BN_F13109719,TTTGTTGGTTCAAGCA,singlet,0,HCA_BN_F13109719,0-HCA_BN_F13109719,NaN,NaN,NaN,...,MaturingChon,11.2,2372.0,skullbase,HCA_BN_F13109719,2372.0,skullbase,11.2,2372.0,11.2


In [18]:
dmx_barcodes = dmx_barcodes.astype(str)


In [19]:
print(dmx_barcodes.dtypes)


Unnamed: 0             object
Cell_ID                object
barcode                object
status                 object
assignment             object
Sample_ID              object
Donor                  object
Sample_ID_pcw13        object
Donor_ID               object
pcw                    object
Sample_ID_published    object
Celltype_fig1          object
pcw_published          object
brc_code               object
anatomical_site        object
Sample_ID_old_anno     object
Donor_old_anno         object
ANATOMICAL_SITE        object
PCW                    object
Donor_final            object
Age_final              object
dtype: object


In [20]:
adata.obs["Donor_Final"]= adata.obs["Donor_ID"].astype(str).copy()
adata.obs["Age_Final"]= adata.obs["Age"].astype(str).copy()

In [21]:
# 1) Ensure your adata.obs index is Cell_ID
#    If not, first:
# adata.obs.set_index('Cell_ID', inplace=True)

# 2) Make sure dmx_barcodes is indexed by Cell_ID as well
dmx = dmx_barcodes.set_index('Cell_ID')

# 3) Find the intersection of indices
common = adata.obs.index.intersection(dmx.index)

# 4) Assign Age_final into Donor_final for those cells
adata.obs.loc[common, 'Donor_Final'] = dmx.loc[common, 'Donor_final']
adata.obs.loc[common, 'Age_Final'] = dmx.loc[common, 'Age_final']

# 5) (Optional) Check a few
print(adata.obs.loc[common[:5], ['Donor_Final']])


                                  Donor_Final
Cell_ID                                      
GTAGGAGGTTTGTTGC-HCA_BN_F13177434      2372.0
ACGCCACAGCTTTGTT-HCA_BN_F13177434      2372.0
GTTTACCGTATTACCC-HCA_BN_F13177434      2372.0
TTCCCGCCAAATACCT-HCA_BN_F13177434      2372.0
GACGCCTAGTTGCCTC-HCA_BN_F13177434      2372.0


In [24]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_filtered_with_metadata_scvi.h5ad')

In [25]:
#adata= sc.read_h5ad('/lustre/scratch126/cellgen/haniffa/sm54/BoneAtlasProject/data/bone_atlas_filtered_with_metadata_scvi.h5ad')

In [25]:
#dmx_barcodes= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/supercell_results/skull_barcodes.csv')

# Build lookup Series
status_map     = dmx_barcodes.set_index('Cell_ID')['status']
assignment_map = dmx_barcodes.set_index('Cell_ID')['assignment']
brc_map = dmx_barcodes.set_index('Cell_ID')['brc_code']
pcw_map = dmx_barcodes.set_index('Cell_ID')['pcw_published']
age_map = dmx_barcodes.set_index('Cell_ID')['Age_final']
donor_map = dmx_barcodes.set_index('Cell_ID')['Donor_final']

# Map into obs and fill missing with "singlet"
adata.obs['dmx_status']     = (
    adata.obs['Cell_ID']
        .map(status_map)
        .fillna('NA')
)
adata.obs['dmx_assignment'] = (
    adata.obs['Cell_ID']
        .map(assignment_map)
        .fillna('NA')
)

adata.obs['dmx_brc_code'] = (
    adata.obs['Cell_ID']
        .map(brc_map)
        .fillna('NA')
)

adata.obs['dmx_pcw_published'] = (
    adata.obs['Cell_ID']
        .map(pcw_map)
        .fillna('NA')
)

adata.obs['dmx_age_final'] = (
    adata.obs['Cell_ID']
        .map(age_map)
        .fillna('NA')
)

adata.obs['dmx_donor_final'] = (
    adata.obs['Cell_ID']
        .map(donor_map)
        .fillna('NA')
)

In [26]:
adata.obs['dmx_status']=adata.obs['dmx_status'].astype(str)
adata.obs['dmx_assignment']=adata.obs['dmx_assignment'].astype(str)
adata.obs['dmx_brc_code']=adata.obs['dmx_brc_code'].astype(str)
adata.obs['dmx_pcw_published'] =adata.obs['dmx_pcw_published'].astype(str)
adata.obs['dmx_age_final']=adata.obs['dmx_age_final'].astype(str)
adata.obs['dmx_donor_final']=adata.obs['dmx_age_final'].astype(str)

In [27]:
adata.obs['dmx_status'].value_counts()

dmx_status
NA            1437283
singlet        125859
doublet         14553
unassigned       1513
Name: count, dtype: int64

In [28]:
adata.obs['dmx_donor_final'].value_counts()

dmx_donor_final
NA      1437283
11.2      38541
13.0      23702
7.3       20765
10.3      15514
8.6       14333
5.8       10764
7.6        9963
8.3        8343
Name: count, dtype: int64

In [29]:
list(adata.obs['Donor_Final'].unique())

['2361',
 '2384',
 'HDBRXXX',
 '2372.0',
 '2365.0',
 '2356/2372',
 '2369.0',
 '2370.0',
 '2370/2369',
 '2393',
 '2394',
 '2382',
 '2372',
 'multiplexed',
 '2394.0',
 '2392.0',
 '2392/2394',
 '2368.0',
 '2392',
 '2358',
 '2370',
 '2369',
 '2365',
 '1111.0',
 '2372/16403',
 'HDBR15918',
 'HDBR15868',
 'HDBR15961',
 'HDBR15916',
 'HDBR15837',
 'HDBR15982',
 'HDBR15927',
 'HDBR15948',
 'GSM4274188',
 'GSM4274189',
 'GSM4274190',
 'GSM4274191',
 'GSM4274192',
 'GSM4274193',
 'GSM4274194',
 'GSM4609136',
 'BRC¬¨‚Ä†2078.00',
 'BRC¬¨‚Ä†2131.00',
 'BRC¬¨‚Ä†2114.00',
 'BRC¬¨‚Ä†2092.00',
 'BRC¬¨‚Ä†2091.00',
 'BRC¬¨‚Ä†2083.00',
 'BRC¬¨‚Ä†2181.00',
 'BRC¬¨‚Ä†2172.00',
 'BRC¬¨‚Ä†2173.00',
 'BRC¬¨‚Ä†2254.00',
 'BRC¬¨‚Ä†2243.00',
 'SB19PCW',
 'F51',
 'F50',
 'F45',
 'F41',
 'F38',
 'F30',
 'F29',
 'F21',
 'C30',
 'C31',
 'C32']

In [30]:
list(dmx_barcodes['Donor_final'].unique())

['2365.0',
 '2372.0',
 '2369.0',
 '2370.0',
 '2392.0',
 '2394.0',
 '2368.0',
 '1111.0']

In [31]:
# 0) Grab your two Series
s1 = dmx_barcodes['Donor_final'].astype(str)
s2 = adata.obs['Donor_Final'].astype(str)

# 1) Define a cleaning function
def clean_label(s: pd.Series) -> pd.Series:
    # a) Decode mojibake: interpret as latin1 → utf-8
    s = s.str.encode('latin1', errors='ignore') \
         .str.decode('utf-8', errors='ignore')
    # b) Drop trailing .0 on numeric labels
    s = s.str.replace(r'\.0$', '', regex=True)
    # # c) Replace any run of non‐alphanumeric with a single hyphen
    # s = s.str.replace(r'[^A-Za-z0-9]+', '-', regex=True)
    # # d) Strip hyphens off the ends
    s = s.str.strip('-')
    return s

# 2) Clean both
dmx_barcodes['Donor_clean'] = clean_label(s1)
adata.obs['Donor_clean']    = clean_label(s2)

# 3) See how many unique labels you now have
print("DMX uniques:", sorted(dmx_barcodes['Donor_clean'].unique()))
print("AnnData uniques:", sorted(adata.obs['Donor_clean'].unique()))

# 4) Find labels in DMX not in AnnData (and vice versa)
dmx_only = set(dmx_barcodes['Donor_clean']) - set(adata.obs['Donor_clean'])
adata_only = set(adata.obs['Donor_clean']) - set(dmx_barcodes['Donor_clean'])
print("In DMX not in AnnData:", dmx_only)
print("In AnnData not in DMX:", adata_only)


DMX uniques: ['1111', '2365', '2368', '2369', '2370', '2372', '2392', '2394']
AnnData uniques: ['1111', '2356/2372', '2358', '2361', '2365', '2368', '2369', '2370', '2370/2369', '2372', '2372/16403', '2382', '2384', '2392', '2392/2394', '2393', '2394', 'BRC2078.00', 'BRC2083.00', 'BRC2091.00', 'BRC2092.00', 'BRC2114.00', 'BRC2131.00', 'BRC2172.00', 'BRC2173.00', 'BRC2181.00', 'BRC2243.00', 'BRC2254.00', 'C30', 'C31', 'C32', 'F21', 'F29', 'F30', 'F38', 'F41', 'F45', 'F50', 'F51', 'GSM4274188', 'GSM4274189', 'GSM4274190', 'GSM4274191', 'GSM4274192', 'GSM4274193', 'GSM4274194', 'GSM4609136', 'HDBR15837', 'HDBR15868', 'HDBR15916', 'HDBR15918', 'HDBR15927', 'HDBR15948', 'HDBR15961', 'HDBR15982', 'HDBRXXX', 'SB19PCW', 'multiplexed']
In DMX not in AnnData: set()
In AnnData not in DMX: {'GSM4274193', 'BRC2131.00', 'BRC2083.00', 'GSM4274192', 'C30', 'BRC2092.00', 'multiplexed', 'F30', 'C31', 'BRC2091.00', '2384', 'HDBR15961', 'F21', 'HDBR15837', '2361', 'BRC2254.00', 'F29', '2392/2394', '2358',

In [32]:
adata.obs['Donor_clean'].value_counts() 

Donor_clean
HDBR15948      264457
2384           204557
HDBRXXX         89929
2372            71364
HDBR15982       65040
2382            52687
HDBR15961       52656
HDBR15837       45632
C32             38477
2365            37356
BRC2243.00      36412
2394            35212
2393            34514
F51             30801
2392            29359
HDBR15868       28986
HDBR15918       26380
BRC2254.00      26266
2358            24771
1111            23702
SB19PCW         22026
C30             20743
F30             19181
F50             17991
F29             17454
2361            17125
F41             16434
F45             15225
2370            14126
2369            13339
C31             12761
BRC2131.00      12553
GSM4274190      11128
BRC2172.00      10536
HDBR15916       10533
BRC2083.00      10474
BRC2181.00       9563
multiplexed      9110
BRC2092.00       8638
BRC2173.00       8359
2368             8343
GSM4274189       8256
GSM4274193       7019
GSM4274192       6851
F38              664

In [36]:
#adata.obs['Donor_ID'] = adata.obs['Donor_ID'].astype(str).str.replace(r'\.0$', '', regex=True)


In [33]:
to_remove= ['2356/2372','2370/2369','2392/2394','multiplexed','2372/16403']

In [34]:
adata=adata[~adata.obs['Donor_clean'].isin(to_remove)]

In [35]:
adata.obs["Donor_clean"].value_counts()

Donor_clean
HDBR15948     264457
2384          204557
HDBRXXX        89929
2372           71364
HDBR15982      65040
2382           52687
HDBR15961      52656
HDBR15837      45632
C32            38477
2365           37356
BRC2243.00     36412
2394           35212
2393           34514
F51            30801
2392           29359
HDBR15868      28986
HDBR15918      26380
BRC2254.00     26266
2358           24771
1111           23702
SB19PCW        22026
C30            20743
F30            19181
F50            17991
F29            17454
2361           17125
F41            16434
F45            15225
2370           14126
2369           13339
C31            12761
BRC2131.00     12553
GSM4274190     11128
BRC2172.00     10536
HDBR15916      10533
BRC2083.00     10474
BRC2181.00      9563
BRC2092.00      8638
BRC2173.00      8359
2368            8343
GSM4274189      8256
GSM4274193      7019
GSM4274192      6851
F38             6640
F21             6474
GSM4609136      6143
GSM4274188      5501
B

In [37]:
adata.write_h5ad('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/bone_atlas_filtered_with_metadata_scvi.h5ad')

/nfs/users/nfs_s/sm54/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/software/cellgen/team298/sm54/envs/boneatlas/lib/python3.9/contextlib.py:126: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/nfs/users/nfs_s/sm54/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/nfs/users/nfs_s/sm54/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/nfs/users/nfs_s/sm54/.local/lib/python3.9/site-

In [36]:
adata.obs.to_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/cell_metadata/bone_atlas_cell_metadata.csv')

In [7]:
import pandas as pd 

In [8]:
doubs= pd.read_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/combined_scrublet_scores_filtered_bone_atlas.csv')

/tmp/ipykernel_1011167/2878560928.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  doubs= pd.read_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/combined_scrublet_scores_filtered_bone_atlas.csv')


In [10]:
doubs['predicted_doublets'].value_counts()

predicted_doublets
False    1396404
True      158025
Name: count, dtype: int64

In [12]:
doubs['predicted_doublets_pval'].value_counts()

predicted_doublets_pval
False    1546870
True       32255
Name: count, dtype: int64

In [1]:
32255 + 1546870

1579125